# Interesting or helpful functions and settings used to interact with PQSC
# But it needs to be noted that the PQSC FW is the most latest one.

In [1]:
import numpy as np
from zhinst import ziPython
import zhinst.ziPython as zi

In [1]:
PORT = 8004
INTERFACE = '1GbE'
SERVER = '10.42.0.66'


# Server connection
daq2 = zi.ziDAQServer(SERVER, PORT, 5)
daq2.setDebugLevel(0)


UHFQA = 'dev2171'
daq2.connectDevice(UHFQA, INTERFACE)

NameError: name 'zi' is not defined

In [8]:
PORT = 8004
INTERFACE = '1GbE'
SERVER = '10.42.0.242'


# Server connection
daq = zi.ziDAQServer(SERVER, PORT, 5)
daq.setDebugLevel(0)

In [9]:
# Device names
HDAWG = 'dev8147'
UHFQA = 'dev2171'
PQSC = 'dev10006'

In [10]:
daq.connectDevice(HDAWG, INTERFACE)
daq.connectDevice(UHFQA, INTERFACE)
daq.connectDevice(PQSC, INTERFACE)

## The PQSC settings: 1 run --> the whole feedback experiment. Both UHF and PQSC used internal clocks although. The holdoff chosen was 100.000 us, with one trigger doing the whole experiment.

### I have to change Elisa's script according to the following. The triggering is completely changed. The time it takes from QA Result finish trigger --> HD-AWG was around 200 ns. 

In [12]:
# UHFQA trigger settings

# Scope module settings
daq.setInt('/dev2171/scopes/0/trigchannel', 2) # Trig chanel chosen is Trigger Input 1
daq.setDouble('/dev2171/scopes/0/trigreference', 0) # No trig reference needed
daq.setDouble('/dev2171/scopes/0/trigdelay', 0) # No trig delay
daq.setDouble('/dev2171/scopes/0/trigholdoff', 2e-05) # This is the lowest value
daq.setDouble('/dev2171/scopes/0/trighysteresis/absolute', 0)
daq.setInt('/dev2171/scopes/0/trigslope', 1)

# Trigger channel
daq.setInt('/dev2171/triggers/out/0/drive', 1) # Probably don't need to drive it?
daq.setInt('/dev2171/triggers/out/0/source', 74) # Choose QA Result Trigger
daq.setDouble('/dev2171/triggers/in/0/level', 0.1) 
daq.setInt('/dev2171/triggers/in/0/imp50', 0) # Need this off - why I'm not 100% sure

In [13]:
# Set strobe of DIO to 'Off': strobe is the signal edge which actives the strobe trigger
# The strobe signal coming from the HDAWG can be set by the AWG - but it does not have to be set as it does not matter
daq.setInt(f'/{UHFQA}/awgs/0/dio/strobe/slope', 0)

# Sets UHFQA DIO mode to QA Result: which are pins DIO[1:11] known as CW
daq.setInt(f'/{UHFQA}/dios/0/mode', 0x2)

# The signals for (QA Result mode DIO) will be driven by bits 16-31
daq.setInt(f'/{UHFQA}/dios/0/drive', 0x3)

# Set sampling rate of DIO data to 50 MHz (?)
daq.setInt(f'/{UHFQA}/dios/0/extclk', 0x2)

# Polarity which is set by DIO[16] must be high for a trigger to occur
daq.setInt(f'/{UHFQA}/awgs/0/dio/valid/polarity', 2)
daq.setInt(f'/{UHFQA}/awgs/0/dio/valid/index', 16)

## This is the crux of the code. Initially I thought 0x004 works. But actually this was the setting that somehow made it all work.

In [14]:
## This will reset the constant HDAWG DIO output which ends up triggering the UHF-QA first
daq.setInt(f'/{HDAWG}/raw/dios/0/mode', 0x004)

# Set the mode to 1 for QCCS mode, 0x042 for old mode
# DIO mux (old style configuration):
# 1st nibble (DIO-out): 2 = ZSync-in to DIO-out (1 = setDIO to DIO-out, 0 = user-val to DIO-out)
# 2nd nibble (AWG-DIO-in): 4 = ZSync-in to waitDIOTrigger (2 = DIO-in to waitDIOTrigger, 1 = "2"+"4")
# 3rd nibble (ZSync-out): 0 = ? (1 = DIO-in to ZSync-out, 2 = setDIO to ZSync-out)
#daq.setInt(f'/{HDAWG}/raw/dios/0/mode', 0x042) # Configured it
## This is the config that has worked for me
daq.setInt(f'/{HDAWG}/raw/dios/0/mode', 0x004)

# The DIO at the master (?) HD-AWG is sampled at 50 MHz
daq.setInt(f'/{HDAWG}/raw/dios/0/extclk', 0x1)

# The HDAWG will drive the DIO bits [16] to [31]
daq.setInt(f'/{HDAWG}/dios/0/drive', 0xc)

# Set strobe of HDAWG to 'Off': strobe is the signal edge which actives the strobe trigger
# The strobe signal coming from the UHFQA is DIO[15] which is always flickering at 25 MHz
daq.setInt(f'/{HDAWG}/awgs/0/dio/strobe/slope', 0)

# Below settings are currently needed for HDAWG triggering from PQSC over ZSync. May change!
daq.setInt(f'/{HDAWG}/awgs/0/dio/valid/polarity', 2)
daq.setInt(f'/{HDAWG}/awgs/0/dio/valid/index', 0)

## The new AWG scripts: DON'T FORGET THAT THE HDAWG SCRIPT IS ALREADY OK, SINCE SHE CALLS THE VARIABLES IN FOR THE WAVEFORMS

In [1]:
UHFawg = '''
var c = 0;

repeat(10){

  setID(1);
  waitDIOTrigger();
  startQAResult();
  
  
  

  
  waitQAResultTrigger();
  //wait(6006); // JS: to account for the 20 us hold-off time
  //c = c + 1;
}

setUserReg(0, c);

'''

In [ ]:
HDawg = '''
    var c = 0; // JS: DEBUG
setUserReg(0, c); // JS: DEBUG

const AWG_N = 32;
const seq_wait = 32;
const gap_wait = 32;
wave w = drag(AWG_N, AWG_N/2, AWG_N/8); // gaussian pulse with (length, center, width)
wave w0 = zeros((seq_wait));
wave w1 = zeros((gap_wait));

repeat(10){
    waitDIOTrigger(); // the trigger coming from the ZSync input, together with the settings in initialize_awg
    //wait(2); // JS: Commented out 
    waitDIOTrigger();
    //c = c+1; // JS: DEBUG

    // getDIOTriggered();
    playWave(w0);
    playWave(1,w);
    playWave(w1);
    playWave(2,w);
    // setUserReg(0, (getDIOTriggered() >> 1) & 0x3ff); // setting the mask to the qubit results
    }
setUserReg(0, c); // JS: DEBUG, reads out number of triggers acquired
'''

## This is a cool (TBD function) I could use to debug how fast the data from the UHF is being recieved in the HDAWG using the Pulse Counter tab in HDAWG

In [15]:
# Debug the frequency of a DIO port for eg. to find out how long the UHF-QA integration path takes to reach HDAWG DIO
# We use the Pulse Counter tab

# Need to set HD master DIO mode to 0 for normal DIO to DIO mode - or 1?


DIObit = 3
daq.setInt(f'/{HDAWG}/cnts/0/enable', 1)
daq.setInt(f'/{HDAWG}/cnts/0/mode', 4) # 4 --> Time tagging: detects a pulse individually and finds the period. Does it over and over for 1 second, and gives us the maximum rate this data was sent
daq.setInt(f'/{HDAWG}/cnts/0/trigfalling', 0) # Detect on trigger rises

freqOfDIO = daq.getInt(f'/{HDAWG}/cnts/0/value')
timePeriodns = (1/freqOfDIO)*1e9
print(f'Frequency of DIO bit {DIObit} is {freqOfDIO}')
print(f'Time taken to recieve trigger value of DIO bit {DIObit} is {timePeriodns} ns')

Frequency of DIO bit 3 is 23310662
Time taken to recieve trigger value of DIO bit 3 is 42.89882458078625 ns
